In [10]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 2, 2
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()


In [11]:
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_4 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

E:\Anaconda\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.493516, acc.: 60.94%] [G loss: 0.616450]
2 [D loss: 0.379568, acc.: 76.56%] [G loss: 0.774427]
3 [D loss: 0.355434, acc.: 71.88%] [G loss: 0.915362]
4 [D loss: 0.233634, acc.: 95.31%] [G loss: 1.046034]
5 [D loss: 0.218598, acc.: 96.88%] [G loss: 1.286552]
6 [D loss: 0.215331, acc.: 96.88%] [G loss: 1.291825]
7 [D loss: 0.178388, acc.: 100.00%] [G loss: 1.444233]
8 [D loss: 0.169896, acc.: 100.00%] [G loss: 1.683358]
9 [D loss: 0.151442, acc.: 100.00%] [G loss: 1.634098]
10 [D loss: 0.121778, acc.: 100.00%] [G loss: 1.900560]
11 [D loss: 0.106019, acc.: 100.00%] [G loss: 1.946505]
12 [D loss: 0.117426, acc.: 100.00%] [G loss: 2.070528]
13 [D loss: 0.092385, acc.: 100.00%] [G loss: 2.105279]
14 [D loss: 0.093702, acc.: 100.00%] [G loss: 2.195267]
15 [D loss: 0.086589, acc.: 100.00%] [G loss: 2.368191]
16 [D loss: 0.076598, acc.: 100.00%] [G loss: 2.350134]
17 [D loss: 0.063455, acc.: 100.00%] [G loss: 2.457372]
18 [D loss: 0.061985, acc.: 100.00%] [G loss: 2.525402]
19 [D l

150 [D loss: 0.514948, acc.: 82.81%] [G loss: 2.933154]
151 [D loss: 0.279003, acc.: 85.94%] [G loss: 4.116714]
152 [D loss: 0.072187, acc.: 98.44%] [G loss: 4.402875]
153 [D loss: 0.355897, acc.: 85.94%] [G loss: 3.608538]
154 [D loss: 0.141731, acc.: 95.31%] [G loss: 4.226253]
155 [D loss: 0.615565, acc.: 71.88%] [G loss: 2.348826]
156 [D loss: 0.273093, acc.: 84.38%] [G loss: 3.952232]
157 [D loss: 0.127383, acc.: 100.00%] [G loss: 3.703529]
158 [D loss: 0.241346, acc.: 90.62%] [G loss: 4.280348]
159 [D loss: 0.179375, acc.: 90.62%] [G loss: 3.742965]
160 [D loss: 0.174776, acc.: 92.19%] [G loss: 3.140528]
161 [D loss: 0.114881, acc.: 95.31%] [G loss: 4.371564]
162 [D loss: 0.485816, acc.: 78.12%] [G loss: 2.694802]
163 [D loss: 0.193983, acc.: 89.06%] [G loss: 4.379726]
164 [D loss: 0.119879, acc.: 96.88%] [G loss: 3.920121]
165 [D loss: 0.109053, acc.: 92.19%] [G loss: 4.105754]
166 [D loss: 0.181241, acc.: 93.75%] [G loss: 3.355665]
167 [D loss: 0.128269, acc.: 96.88%] [G loss: 3

297 [D loss: 0.817035, acc.: 45.31%] [G loss: 0.791219]
298 [D loss: 0.716620, acc.: 50.00%] [G loss: 0.802782]
299 [D loss: 0.694365, acc.: 51.56%] [G loss: 0.924038]
300 [D loss: 0.783780, acc.: 37.50%] [G loss: 0.806840]
301 [D loss: 0.757168, acc.: 48.44%] [G loss: 0.858216]
302 [D loss: 0.714921, acc.: 50.00%] [G loss: 0.872980]
303 [D loss: 0.776319, acc.: 40.62%] [G loss: 0.797431]
304 [D loss: 0.730835, acc.: 53.12%] [G loss: 0.805096]
305 [D loss: 0.715353, acc.: 54.69%] [G loss: 0.872241]
306 [D loss: 0.763284, acc.: 43.75%] [G loss: 0.868013]
307 [D loss: 0.788258, acc.: 42.19%] [G loss: 0.747631]
308 [D loss: 0.740165, acc.: 43.75%] [G loss: 0.809571]
309 [D loss: 0.671822, acc.: 53.12%] [G loss: 0.883232]
310 [D loss: 0.839341, acc.: 26.56%] [G loss: 0.668471]
311 [D loss: 0.733126, acc.: 50.00%] [G loss: 0.708283]
312 [D loss: 0.720021, acc.: 43.75%] [G loss: 0.724617]
313 [D loss: 0.770118, acc.: 37.50%] [G loss: 0.638412]
314 [D loss: 0.720537, acc.: 48.44%] [G loss: 0.

...

17557 [D loss: 0.693283, acc.: 60.94%] [G loss: 0.901516]
17558 [D loss: 0.495424, acc.: 76.56%] [G loss: 0.986359]
17559 [D loss: 0.602245, acc.: 62.50%] [G loss: 0.995306]
17560 [D loss: 0.620459, acc.: 60.94%] [G loss: 1.002090]
17561 [D loss: 0.681973, acc.: 53.12%] [G loss: 0.856216]
17562 [D loss: 0.708555, acc.: 54.69%] [G loss: 0.954875]
17563 [D loss: 0.618428, acc.: 57.81%] [G loss: 1.006240]
17564 [D loss: 0.705637, acc.: 56.25%] [G loss: 0.976405]
17565 [D loss: 0.654931, acc.: 57.81%] [G loss: 0.946259]
17566 [D loss: 0.663122, acc.: 59.38%] [G loss: 0.942166]
17567 [D loss: 0.555289, acc.: 78.12%] [G loss: 0.985358]
17568 [D loss: 0.611324, acc.: 70.31%] [G loss: 1.049160]
17569 [D loss: 0.678511, acc.: 67.19%] [G loss: 0.927239]
17570 [D loss: 0.615792, acc.: 67.19%] [G loss: 0.945462]
17571 [D loss: 0.596049, acc.: 68.75%] [G loss: 1.021187]
17572 [D loss: 0.567266, acc.: 71.88%] [G loss: 0.979485]
17573 [D loss: 0.580731, acc.: 67.19%] [G loss: 0.936254]
17574 [D loss:

17699 [D loss: 0.654993, acc.: 57.81%] [G loss: 0.953334]
17700 [D loss: 0.593767, acc.: 68.75%] [G loss: 0.987037]
17701 [D loss: 0.644280, acc.: 64.06%] [G loss: 0.965395]
17702 [D loss: 0.642861, acc.: 56.25%] [G loss: 0.977908]
17703 [D loss: 0.621592, acc.: 62.50%] [G loss: 1.001454]
17704 [D loss: 0.697576, acc.: 53.12%] [G loss: 0.913169]
17705 [D loss: 0.611320, acc.: 68.75%] [G loss: 1.018750]
17706 [D loss: 0.627999, acc.: 65.62%] [G loss: 0.977761]
17707 [D loss: 0.555056, acc.: 70.31%] [G loss: 1.118918]
17708 [D loss: 0.632860, acc.: 64.06%] [G loss: 1.101510]
17709 [D loss: 0.654458, acc.: 62.50%] [G loss: 0.944446]
17710 [D loss: 0.743610, acc.: 50.00%] [G loss: 1.051015]
17711 [D loss: 0.561019, acc.: 71.88%] [G loss: 0.934499]
17712 [D loss: 0.616336, acc.: 65.62%] [G loss: 0.957902]
17713 [D loss: 0.683941, acc.: 56.25%] [G loss: 1.005380]
17714 [D loss: 0.610765, acc.: 70.31%] [G loss: 1.075486]
17715 [D loss: 0.641921, acc.: 59.38%] [G loss: 0.973848]
17716 [D loss:

17843 [D loss: 0.641595, acc.: 54.69%] [G loss: 0.947039]
17844 [D loss: 0.689881, acc.: 53.12%] [G loss: 0.922524]
17845 [D loss: 0.586815, acc.: 70.31%] [G loss: 0.835686]
17846 [D loss: 0.621258, acc.: 68.75%] [G loss: 0.976653]
17847 [D loss: 0.637833, acc.: 67.19%] [G loss: 0.875341]
17848 [D loss: 0.654778, acc.: 62.50%] [G loss: 1.011837]
17849 [D loss: 0.589448, acc.: 71.88%] [G loss: 1.039680]
17850 [D loss: 0.637186, acc.: 56.25%] [G loss: 0.945687]
17851 [D loss: 0.636271, acc.: 62.50%] [G loss: 0.921508]
17852 [D loss: 0.669601, acc.: 56.25%] [G loss: 0.907003]
17853 [D loss: 0.652337, acc.: 64.06%] [G loss: 0.942299]
17854 [D loss: 0.631418, acc.: 67.19%] [G loss: 0.993513]
17855 [D loss: 0.673568, acc.: 57.81%] [G loss: 1.033491]
17856 [D loss: 0.707687, acc.: 57.81%] [G loss: 1.056201]
17857 [D loss: 0.783985, acc.: 50.00%] [G loss: 0.996529]
17858 [D loss: 0.633049, acc.: 65.62%] [G loss: 1.106563]
17859 [D loss: 0.638493, acc.: 60.94%] [G loss: 0.972557]
17860 [D loss:

17986 [D loss: 0.689088, acc.: 51.56%] [G loss: 0.952832]
17987 [D loss: 0.652238, acc.: 68.75%] [G loss: 1.018373]
17988 [D loss: 0.588958, acc.: 68.75%] [G loss: 0.979307]
17989 [D loss: 0.572689, acc.: 73.44%] [G loss: 0.923348]
17990 [D loss: 0.616787, acc.: 67.19%] [G loss: 1.084424]
17991 [D loss: 0.572070, acc.: 71.88%] [G loss: 1.064533]
17992 [D loss: 0.676990, acc.: 62.50%] [G loss: 1.053242]
17993 [D loss: 0.620483, acc.: 67.19%] [G loss: 1.049769]
17994 [D loss: 0.664636, acc.: 57.81%] [G loss: 0.981155]
17995 [D loss: 0.649996, acc.: 57.81%] [G loss: 0.968840]
17996 [D loss: 0.674703, acc.: 60.94%] [G loss: 1.054587]
17997 [D loss: 0.713188, acc.: 51.56%] [G loss: 0.998334]
17998 [D loss: 0.657472, acc.: 59.38%] [G loss: 0.884082]
17999 [D loss: 0.624562, acc.: 70.31%] [G loss: 0.901223]
18000 [D loss: 0.675827, acc.: 59.38%] [G loss: 0.929609]
18001 [D loss: 0.670729, acc.: 59.38%] [G loss: 0.837045]
18002 [D loss: 0.611358, acc.: 70.31%] [G loss: 0.959988]
18003 [D loss:

KeyboardInterrupt: 